# 6.2 Стекинг

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [24]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [25]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [26]:
df.shape

(10000, 55)

In [27]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [28]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [29]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [30]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора.     Они являются вероятностями классов при решении задачи многоклассовой классификации.
    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [31]:
def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.

    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [32]:
np.random.seed(42)

In [33]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.795630725863284

In [34]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:15<00:00,  3.98s/it]


In [35]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [36]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.8033826638477801

# Задания


In [218]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)



# Задание 6.6.1

В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.


Логика решения:
1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);
3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.



In [217]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.     Они являются вероятностями классов при решении задачи многоклассовой классификации.    :arg clf: классификатор    :args X_train, y_train: обучающая выборка    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [149]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора.
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
# Напишите ваш код ниже

Для следующих заданий используйте этот код:

In [208]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):

    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [209]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):

    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]

    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])


    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])

    return stacked_features_train, stacked_features_test

In [210]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

# Задание 6.6.2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
случайный лес из 300 деревьев
градиентный бустинг из 200 деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.


In [191]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr'),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', max_iter=2000, multi_class='multinomial'),
    RandomForestClassifier(n_estimators=300),
    GradientBoostingClassifier(n_estimators=200)
], X_train, X_test, y_train, cv)


100%|██████████| 4/4 [01:52<00:00, 28.16s/it]


In [192]:
compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)




0.984254

# Задание 6.6.3

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев
случайный лес из 200 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [185]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    GradientBoostingClassifier(n_estimators=200)
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [01:47<00:00, 53.91s/it]


In [193]:
compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)


0.984254

# Задание 6.6.4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
метод ближайшего соседа (k-NN) со стандартными параметрами
случайный лес из 300 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [200]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

X_meta_train, X_meta_test = generate_meta_features([
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300)
], X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:05<00:00,  2.93s/it]


In [201]:

compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)


C:\PythonProject\skillfactory мат анализ\.venv1\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.990302

# Задание 6.6.5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
метод ближайшего соседа со стандартными параметрами
случайный лес из 300 экстремальных деревьев
AdaBoost со стандартными параметрами
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [202]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

X_meta_train, X_meta_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr'),
    KNeighborsClassifier(),
    AdaBoostClassifier()
], X_train, X_test, y_train, cv)

100%|██████████| 3/3 [00:37<00:00, 12.40s/it]


In [203]:

compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)

C:\PythonProject\skillfactory мат анализ\.venv1\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.983745

Задание 6.6.6

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
1) случайный лес из 300 деревьев
2) случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [222]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300),
], X_train_scaler, X_test_scaler, y_train, cv)



100%|██████████| 2/2 [00:13<00:00,  6.55s/it]


In [223]:
compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)


C:\PythonProject\skillfactory мат анализ\.venv1\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.983963

# Задание 6.6.7

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [224]:
cv = KFold(n_splits=20, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300),
], X_train, X_test, y_train, cv)



100%|██████████| 2/2 [00:24<00:00, 12.21s/it]


In [225]:
compute_metric(LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto'), X_meta_train ,y_train, X_meta_test)

C:\PythonProject\skillfactory мат анализ\.venv1\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.982297

# Задание 6.6.8

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [227]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300),
], X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:06<00:00,  3.25s/it]


In [228]:
compute_metric(RandomForestClassifier(), X_meta_train ,y_train, X_meta_test)


0.981669

# Задание 6.6.9

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300),
], X_train, X_test, y_train, cv)


In [229]:
compute_metric(KNeighborsClassifier(), X_meta_train ,y_train, X_meta_test)


0.98417

# Задание 6.6.10

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [230]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300),
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:15<00:00,  7.66s/it]


In [231]:
compute_metric(GradientBoostingClassifier(), X_meta_train ,y_train, X_meta_test)


0.983472

# Задание 6.6.11

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
1. случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
2. случайный лес из 300 экстремальных деревьев
Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

X_meta_train, X_meta_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, criterion='gini', min_samples_split=24),
    RandomForestClassifier(n_estimators=300),
], X_train, X_test, y_train, cv)

In [ ]:
compute_metric(GradientBoostingClassifier(), X_meta_train ,y_train, X_meta_test)